In [15]:
# Initial imports
import os
import pandas as pd
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
# Reading the News API key enviroment variable
api_key = os.getenv("crypto_news_key")

In [17]:
print(f"API key:{type(api_key)}")

API key:<class 'str'>


In [18]:
#data pull is from the last 4 or five days
general_crypto_news_data = 'https://cryptonews-api.com/api/v1/category?section=general&items=50&token='+ api_key


In [19]:
response_news = requests.get(general_crypto_news_data)

In [20]:
response_news

<Response [200]>

In [21]:
news_data_raw = response_news.json()

In [22]:
news_df = pd.DataFrame(data=news_data_raw['data'])

In [23]:
news_df.tail()

news_url  \
45  https://cryptonews.com/news/6-000-xrp-holders-...   
46    https://cryptodaily.co.uk/2021/03/traders-arena   
47  https://beincrypto.com/several-binance-smart-c...   
48  https://dailyhodl.com/2021/03/15/vast-majority...   
49  https://u.today/financial-services-company-dis...   

                                            image_url  \
45  https://crypto.snapi.dev/images/v1/h/e/f9b7d45...   
46  https://crypto.snapi.dev/images/v1/p/r/price-1...   
47  https://crypto.snapi.dev/images/v1/z/x/bic-rec...   
48  https://crypto.snapi.dev/images/v1/l/a/last-mi...   
49  https://crypto.snapi.dev/images/v1/b/j/6805-16...   

                                                title  \
45  6,000+ XRP Holders Want to Testify, Bitcoin St...   
46         There's a New Challenge in the Crypto Area   
47  Several Binance Smart Chain Projects Under DNS...   
48  Vast Majority of Accredited Investors Plan To ...   
49  Financial Services Company Discloses $1 Millio...   

                                                 text     source_name  \
45  Get your daily, bite-sized digest of cryptoass...      Cryptonews   
46  If you are a big fan of Bitcoin trading, then ...    Crypto Daily   
47  A DNS attack can take several forms. However, ...      BeInCrypto   
48  The bulk of accredited investors are consideri...  The Daily Hodl   
49  Another public company now has a six-figure su...          UToday   

                               date topics sentiment     type  
45  Mon, 15 Mar 2021 13:37:00 -0400     []   Neutral  Article  
46  Mon, 15 Mar 2021 13:30:00 -0400     []  Negative  Article  
47  Mon, 15 Mar 2021 12:11:31 -0400     []  Negative  Article  
48  Mon, 15 Mar 2021 12:10:47 -0400     []  Positive  Article  
49  Mon, 15 Mar 2021 12:05:05 -0400     []  Positive  Article

In [24]:
general_recent_news_df = news_df.drop(columns = ['news_url','image_url','source_name'])
general_recent_news_df.head()

title  \
0  Messari CEO thinks a fresh wave of crypto regu...   
1  Digital Dollar Would Have to Coexist With Cash...   
2  Hawaiian House Democrats ask the state regulat...   
3  CBDCs won't entirely replace cash if the US Fe...   
4  Building digital dollar requires private secto...   

                                                text  \
0  After nearly a decade of progress since Bitcoi...   
1  Jerome Powell today said that a central bank d...   
2  Hawaiian House Democrats are requesting the De...   
3  The U.S. does not want cash to go extinct in t...   
4  The chairman of the Federal Reserve said today...   

                              date            topics sentiment     type  
0  Thu, 18 Mar 2021 18:00:00 -0400     [regulations]  Negative  Article  
1  Thu, 18 Mar 2021 17:01:07 -0400  [digital dollar]   Neutral  Article  
2  Thu, 18 Mar 2021 16:59:51 -0400     [regulations]  Positive  Article  
3  Thu, 18 Mar 2021 16:57:22 -0400                []   Neutral  Article  
4  Thu, 18 Mar 2021 16:47:27 -0400  [digital dollar]   Neutral  Article

In [25]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [26]:
# Instantiate the lemmatizer
wnl = WordNetLemmatizer() 

# Create a list of stopwords
stop = stopwords.words('english')

# Expand the default stopwords list if necessary
stop.append("u")
stop.append("it'")
stop.append("'s")
stop.append("n't")
stop.append('…')
stop.append("\`")
stop.append('``')
stop.append('char')
stop.append("''")
stop = set(stop)

In [27]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words
    words = word_tokenize(text)

    # Convert the words to lowercase
    words = list(filter(lambda w: w.lower(), words))
    
    # Remove the punctuation
    words = list(filter(lambda t: t not in punctuation, words))
    
    # Remove the stopwords
    words = list(filter(lambda t: t.lower() not in stop, words))
    
    # Lemmatize Words into root words
    tokens = [wnl.lemmatize(word) for word in words]
    
    return tokens


In [29]:
general_recent_news_df['tokens']=general_recent_news_df.text.apply(tokenizer)
general_recent_news_df.head()

title  \
0  Messari CEO thinks a fresh wave of crypto regu...   
1  Digital Dollar Would Have to Coexist With Cash...   
2  Hawaiian House Democrats ask the state regulat...   
3  CBDCs won't entirely replace cash if the US Fe...   
4  Building digital dollar requires private secto...   

                                                text  \
0  After nearly a decade of progress since Bitcoi...   
1  Jerome Powell today said that a central bank d...   
2  Hawaiian House Democrats are requesting the De...   
3  The U.S. does not want cash to go extinct in t...   
4  The chairman of the Federal Reserve said today...   

                              date            topics sentiment     type  \
0  Thu, 18 Mar 2021 18:00:00 -0400     [regulations]  Negative  Article   
1  Thu, 18 Mar 2021 17:01:07 -0400  [digital dollar]   Neutral  Article   
2  Thu, 18 Mar 2021 16:59:51 -0400     [regulations]  Positive  Article   
3  Thu, 18 Mar 2021 16:57:22 -0400                []   Neutral  Article   
4  Thu, 18 Mar 2021 16:47:27 -0400  [digital dollar]   Neutral  Article   

                                              tokens  
0  [nearly, decade, progress, since, Bitcoin, inc...  
1  [Jerome, Powell, today, said, central, bank, d...  
2  [Hawaiian, House, Democrats, requesting, Depar...  
3  [U.S., want, cash, go, extinct, presence, CBDC...  
4  [chairman, Federal, Reserve, said, today, U.S....